In [ ]:
#########################
# defs and imports
#########################

import logging
import os

import numpy as np
from numpy.linalg import inv

from nis_util import *
from simple_detection import *


def bbox_pix2unit2(bbox, start, pixsize, direction, fov, mat):    
    """
    old, possibly wrong pix2unit
    TODO: remove if no longer necessary
    """
    
    logger = logging.getLogger(__name__)
    
    extra_offset = inv(mat).dot(np.array(list(reversed(fov)), dtype=float)/2) * np.array(list(reversed(direction)), dtype=float)
    start_ = np.array(start, dtype=float) + np.array(list(reversed(list(extra_offset))), dtype=float)
    
    logger.debug('extra offset: {}, new start: {}'.format(extra_offset, start_))
    
    (ymin, xmin, ymax, xmax) = bbox
    
    bbox_tr = np.apply_along_axis(lambda x: mat.dot(x), 1, np.array([[xmin, ymin], 
                                                           [xmin, ymax],
                                                           [xmax, ymin],
                                                           [xmax, ymax]], dtype=float))
    # TODO: WHY?
    bbox_tr *= -1
    
    min_ = np.apply_along_axis(np.min, 0, bbox_tr)
    max_ = np.apply_along_axis(np.max, 0, bbox_tr)
    
    logger.debug('new min: {}, new max: {}'.format(min_, max_))
    
    bbox_tr_arr = np.array([list(reversed(list(min_))), list(reversed(list(max_)))], dtype=float)
    
    res = (bbox_tr_arr * np.array(pixsize, dtype=float) *
            np.array(direction, dtype=float) + start_)
    
    logger.debug('bbox: {}, toUnit: {}'.format(bbox, res.reshape((4,))))
    return res.reshape((4,))


def do_scan(field, oc_overview, ocs_detail, path_to_nis, save_base_path, prefix, suffix='.nd2', do_plot=True,
           z_range = 10, z_step=2, z_drive='Ti2 ZDrive', dry_run_details=False, stitched=True):
    
    logger = logging.getLogger(__name__)
    
    if stitched and not np.isscalar(ocs_detail):
        logger.warn('Doing multi-channel acquisition, cannot use NIS stitching. Please stitch manually.')
    
    # get field and directions
    # NB: this is not the actual field being scanned, but rather [min+1/2 fov - max-1/2fov]
    (left, right, top, bottom) = field
    direction = [1 if top<bottom else -1, 1 if left<right else -1]
    
    # set overview optical configuration
    set_optical_configuration(path_to_nis, oc_overview)
    
    # get resolution and fov
    (xres, yres, siz, mag) = get_resolution(path_to_nis)
    fov_x = xres * siz / mag
    fov_y = yres * siz / mag
    
    logger.debug('overview resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
    
    # do overview scan
    ov_path = os.path.join(save_base_path, prefix + '_overview' + suffix)    
    do_large_image_scan(path_to_nis, ov_path, left, right, top, bottom)
    
    logger.info('finished overview, detecting wings...')
    
    # detect wings
    img = read_bf(ov_path)    
    bboxes = detect_wings_simple(img, plot=do_plot, layers=2)
    
    # get camera rotation
    mat = np.array(get_rotation_matrix(path_to_nis)).reshape((2,2))
    
    # pixels to units
    bboxes = [bbox_pix2unit2(b, [top, left], [siz/mag,siz/mag], direction,[fov_y, fov_x], mat) for b in bboxes]
    
    # expand bounding boxes
    # bboxes = [scale_bbox(bbox, expand_factor=.15) for bbox in bboxes]
    
    print('detected {} wings:'.format(len(bboxes)))

    for idx, bbox in enumerate(bboxes):
        
        print('scanning wing {}: {}'.format(idx, bbox))
        
        (ymin, xmin, ymax, xmax) = bbox
        wing_path = os.path.join(save_base_path, prefix + '_wing' + str(idx) + suffix) 
        
        # only one optical configuration
        if np.isscalar(ocs_detail):
            
            # set oc so we have correct magnification
            set_optical_configuration(path_to_nis, ocs_detail)

            # get resolution
            (xres, yres, siz, mag) = get_resolution(path_to_nis)
            fov = get_fov_from_res(get_resolution(path_to_nis))
            logger.debug('detail resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
            logger.debug('fov: {}'.format(fov))

            # get fov
            fov_x = xres * siz / mag
            fov_y = yres * siz / mag
            
            # correct for half view offset (we add this time) - may not be neccessary at this magnification/fov
            # we should probably do that, ignore it for now
            # grid = [[g[0] + fov_x * direction[1] / 2.0, g[1] + fov_y * direction [0] / 2.0] for g in grid]
            if not stitched:
                for g in grid:
                    logger.debug('wing {}: will scan tile at {}'.format(idx-1, g))

            # do not actually do the detail acquisition
            if dry_run_details:
                continue
                
            # do a manual grid acquisition via multipoint nD acquisition -> has to be stitched afterwards
            if not stitched:
                
                # we scan around current z -> get that
                pos = get_position(path_to_nis)
                
                # generate the coordinates of the tiles
                grid = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True)
                
                nda = NDAcquisition(wing_path)
                nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
                nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))
                nda.prepare(path_to_nis)
                nda.run(path_to_nis)
                
            # do NIS's scan large image -> stitching is performend in NIS
            else:
                do_large_image_scan(path_to_nis, wing_path, xmax, xmin, ymin, ymax, 15, True)
        
        # multiple ocs -> we have to do nD acquisition
        else:
            
            # set to first oc so we have correct magnification
            set_optical_configuration(path_to_nis, ocs_detail[0])

            # get resolution
            (xres, yres, siz, mag) = get_resolution(path_to_nis)
            fov = get_fov_from_res(get_resolution(path_to_nis))
            logger.debug('detail resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
            logger.debug('fov: {}'.format(fov))

            # get fov
            fov_x = xres * siz / mag
            fov_y = yres * siz / mag
            
            # correct for half view offset (we add this time) - may not be neccessary at this magnification/fov
            # we should probably do that, ignore it for now
            # grid = [[g[0] + fov_x * direction[1] / 2.0, g[1] + fov_y * direction [0] / 2.0] for g in grid]
            if not stitched:
                for g in grid:
                    logger.debug('wing {}: will scan tile at {}'.format(idx-1, g))

            # do not actually do the detail acquisition
            if dry_run_details:
                continue
                
            # NB: we have multiple channels, so we have to do
            # manual grid acquisition via multipoint nD acquisition -> has to be stitched afterwards
                
            # we scan around current z -> get that
            pos = get_position(path_to_nis)
                
            # generate the coordinates of the tiles
            grid = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True)
            
            # setup nD acquisition
            nda = NDAcquisition(wing_path)
            nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
            nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))
            
            for oc in ocs_detail:
                nda.add_c(oc)
            
            nda.prepare(path_to_nis)
            nda.run(path_to_nis)

            

In [ ]:
###################
# set up the environment, nis, and image saving path
###################

path_to_nis = 'C:\\Program Files\\NIS-Elements\\nis_ar.exe'
save_base_path = 'C:\\Users\\Nikon\\Documents\\David\\tmpOverview'

%matplotlib inline

In [ ]:
#################
# do the scans
#################

logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.DEBUG)
logger = logging.getLogger(__name__)

# name of the slides to image
# set to 'None' to skip a slide

slide_left = None #'NG_Overview_009'
slide_mid = None #'NG_Overview_024'
slide_right = None #'NG_Overview_025'

do_scan_left  = slide_left != None
if do_scan_left:
    logger.info('Scanning left scan.')
    do_scan((53331, 28806, -20726, 20464), 'DIA4x', ['DIA10x'], path_to_nis, save_base_path, slide_left, dry_run_details=True )
else:
    logger.info('Skipping left slide.')

# mid slide
do_scan_mid  = slide_mid != None
if do_scan_mid:
    logger.info('Scanning mid scan.')
    do_scan((13400, -7850, -20954, 18220), 'DIA4x', ['DIA10x'], path_to_nis, save_base_path, slide_left )
else:
    logger.info('Skipping middle slide.')

# right slide
do_scan_right  = slide_right != None
if do_scan_right:
    logger.info('Scanning right scan.')
    do_scan((-26500, -50000, -21053, 18177), 'DIA4x', ['DIA10x'], path_to_nis, save_base_path, slide_left )
else:
    logger.info('Skipping right slide.')